In [2]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
import pickle 
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
sc = pyspark.SparkContext()
temp_dir = working_dir = os.path.join(os.environ["HAIL_HOME"], "tmp")
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

/opt/sanger.ac.uk/hgi/anaconda3/lib/python3.7/site-packages/hail/context.py:71: UserWarning: pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jar
  'pip-installed Hail requires additional configuration options in Spark referring\n'
Running on Apache Spark version 2.4.3
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.31-6060f9c971cc
LOGGING: writing to /opt/sanger.ac.uk/hgi/hail/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20200727-1323-0.2.31-6060f9c971cc.log


Loading BokehJS ...

In [3]:
pca_scores=hl.read_table(f"{temp_dir}/ddd-elgh-ukbb/pop_assignments_test.ht")
mt= hl.read_matrix_table(f"{temp_dir}/ddd-elgh-ukbb/Sanger_cohorts_chr1-20-XY_new_cohorts_split_multi.mt")

In [4]:
pca_scores=pca_scores.annotate(cohort=mt.cols()[pca_scores.s].cohort)

2020-07-27 13:24:02 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [5]:
pca_scores.write(f"{temp_dir}/ddd-elgh-ukbb/pop_assignments_cohorts_annotated_AKT_overlap.ht")

2020-07-27 13:24:18 Hail: INFO: Coerced sorted dataset
2020-07-27 13:25:26 Hail: INFO: wrote table with 93674 rows in 512 partitions to /opt/sanger.ac.uk/hgi/hail/tmp/ddd-elgh-ukbb/pop_assignments_cohorts_annotated_AKT_overlap.ht


In [6]:
pca_scores=hl.read_table(f"{temp_dir}/ddd-elgh-ukbb/pop_assignments_cohorts_annotated_AKT_overlap.ht")
pca_scores.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    's': str 
    'known_pop': str 
    'pca_scores': array<float64> 
    'pop': str 
    'prob_African': float64 
    'prob_Any other Asian background': float64 
    'prob_Any other Black background': float64 
    'prob_Any other mixed background': float64 
    'prob_Any other white background': float64 
    'prob_Asian or Asian British': float64 
    'prob_Bangladeshi': float64 
    'prob_Black or Black British': float64 
    'prob_British': float64 
    'prob_Caribbean': float64 
    'prob_Chinese': float64 
    'prob_Do not know': float64 
    'prob_Indian': float64 
    'prob_Irish': float64 
    'prob_Mixed': float64 
    'prob_Other ethnic group': float64 
    'prob_Pakistani': float64 
    'prob_Prefer not to answer': float64 
    'prob_White': float64 
    'prob_White and Asian': float64 
    'prob_White and Black African': float64 
    'prob_White and Black Ca

In [7]:
p1=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[1], xlabel="PC1", ylabel="PC2", label=pca_scores.cohort)
p2=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[2], xlabel="PC1", ylabel="PC3", label=pca_scores.cohort)


show(bokeh.layouts.gridplot([[p1],[p2]]))

In [8]:
p1=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[1], xlabel="PC1", ylabel="PC2", label=pca_scores.pop)
p2=hl.plot.scatter(pca_scores.pca_scores[0], pca_scores.pca_scores[2], xlabel="PC1", ylabel="PC3", label=pca_scores.pop)


show(bokeh.layouts.gridplot([[p1],[p2]]))